# Quantum Generative Adversarial Network

This notebook contains documentation of the algorithm and testing codes. First run the following line to load the package.

In [76]:
%run 'QGAN.py'

## Classes

### Class `Pauli`

`Pauli(Xs, Zs, L)` representes a Pauli (string) operator labeled by
- two sets $\mathcal{X}$ (`Xs`) and $\mathcal{Z}$ (`Zs`)  that specifies the positions on which $\sigma^x$ and $\sigma^z$ act respectively,
- and a integer `L` that specifies the total number of qubits on which the Pauli operator acts.

$$\sigma = \mathrm{i}^{|\mathcal{X}\cap\mathcal{Z}|}\otimes_{i\in\mathcal{X}} \sigma_i^1\otimes_{i\in\mathcal{Z}}\sigma_i^3$$

`Pauli.mat()` returns the matrix representation (in terms of scipy CSR sparse array) of the Pauli operator. We assume all local operators are modeled by Pauli operators.

In [494]:
op = Pauli({1},{0,1}, 2) 
op.mat().toarray(), op

(array([[ 0.+0.j,  0.-1.j,  0.+0.j,  0.+0.j],
        [ 0.+1.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j,  0.+1.j],
        [ 0.+0.j,  0.+0.j,  0.-1.j,  0.+0.j]]), σ[3, 2])

### Class `Model`

`Model(L, loc = 2)` model a system of length `L` (number of qubits) on with local operators supported on `loc` adjacent sites. Periodic boundary condition is assumed.
- `Model.dim` is the many-body Hilbert space dimension.
- `Model.ops` holds the basis of local operators.
- `Model.opdim` is the dimension of the local operator space.
- `Model.sample()` draws a sample of `(h,M)` pair.

A simple one qubit model.

In [78]:
mdl = Model(1)
# Hilbert space dim, operator space dim, operators
mdl.dim, mdl.opdim, mdl.ops

(2, 3, [σ[1], σ[2], σ[3]])

As expected, the ground state of correlation matrix recovers the Hamiltonian nicely.

In [82]:
(h, M) = mdl.sample()
h, GS(M)

(array([-0.55193861, -0.48502894, -0.67831461]),
 array([-0.55193861, -0.48502894, -0.67831461]))

More serious 10-qubit model. Calculate the fidelity between the Hamiltonian and the ground state of the correlation matrix. The fidelity is typically ~ 1.

In [83]:
mdl = Model(10)
(h, M) = mdl.sample()
abs(GS(M).dot(h))

0.99999999999999956

Once the model is built, the calculation of correlation matrix can be faster (~ 100ms to generate a sample of (H,M) pair for 10-qubit system). 

In [85]:
%%time
(h, M) = mdl.sample()

CPU times: user 101 ms, sys: 3.9 ms, total: 105 ms
Wall time: 97.9 ms


### Class `MPS`

`MPS(depth, dim)` creates an MPS of depth `depth` with the bond dimension `dim` (which should match the operator space dimension).
- `MPS.P(h)` get P matrix from Hamiltonian vector `h`.
- `MPS.mul(val)` multiplies the resulting P matrix by `val`.

In [74]:
mps = MPS(3, 2)
mps.mul(2.)
mps.P(torch.ones(2))

Variable containing:
 2.0000  0.0000
 0.0000  2.0000
[torch.FloatTensor of size 2x2]

In [261]:
%run "QGAN.py"
u = MPSunit(2)
u.push(torch.ones(2))
u.mat.data += 0.2*torch.ones(2,2)
u.update()


 1.2000  0.2000
 0.2000  1.2000
[torch.FloatTensor of size 2x2]

In [262]:
u.s


 1.5181
 0.1242
[torch.FloatTensor of size 2]

In [125]:
torch.sqrt(s[:1])


 1.2559
[torch.FloatTensor of size 1]

In [674]:
mps = MPS(5, 2)

Basic functions:
- `MPS.initialize(val)` initializes the MPS to be a two-way multiplier between the left and the right auxilary spaces, with the multiplication factor set by `val`, given the physical legs are pinned to $(1,0,0\cdots)$.
- `MPS.pin(phys_env)` pins all physical legs to the physical environment vector `phys_env`.
- `MPS.fromright(right_env)` takes the right environment vecotor `right_env` from the right end and push it all the way to the left, returning the output on the left end (typically used to pass the random source forward).
- `MPS.fromleft(left_env)` takes the left environment vector `left_env` from the left end and push it all the way to the right, returning the output on the right end (typically used to propagate the gradient backward).

In [675]:
mps.initialize(2.)
mps.pin([1,0,0])
mps.fromleft([1,3])

array([ 2.,  6.])

In our application, we will always take the right auxilary space as the latent space and the left auxilary space as the operator space. Given the physical legs pinned,
- `MPS.evaluate()` contract the right auxilary space and returns the density matrix in the operator space,
- `MPS.update(grad)` pass down the gradient of the density matrix `grad`, the MPS will be updated by ascending the gradient using the stocastic training method. (One should not expect the density matrix to be updated by the exact among of gradient, because the gradient signal is just a force acting on the MPS, how the MPS response to the force with displacement still depends on the stiffness, but different matrix element has different stiffness so the update outcome is onlu roughly along the gradient ascending direction).

In [676]:
mps.evaluate()

array([[ 4.,  0.],
       [ 0.,  4.]])

In [677]:
mps.update(0.01*numpy.array([[0,1],[1,0]]))
mps.evaluate()

array([[ 4.67782278,  1.4174439 ],
       [ 1.4174439 ,  4.85481607]])

Interface functions (with pytorch) for MPS optimization.
- `MPS.getP(h)` given the Hamiltonian (in vector form) `h`, calculate the density matrix by pinning the physical legs to $v=(1,h)$. The latent space is integrated over, s.t.
$$P(h)=MPS(h)\cdot MPS(h)^\intercal.$$
- `MPS.optimize(V)` gradient **descent** the value function `V` by updating the MPS. 

Set a objective $P_0$ to approach:

In [706]:
P0 = torch.autograd.Variable(torch.Tensor(numpy.array(
            [[0.7,0.1],
             [0.1,0.3]])))

The MPS can be simply trained by considering the quadratic loss function. Using the stocastic training approach by sampling random vectors in the latent space, the convergence looks great. 

In [733]:
mps.initialize(1.)
for i in range(200):
    P = mps.getP([0.2,0.4])
    V = torch.sum((P - P0)**2)
    mps.optimize(V)
    if i%50 == 0:
        print(P.data.numpy(), V.data.numpy())

[[ 1.  0.]
 [ 0.  1.]] [ 0.59999996]
[[ 0.77360308  0.15239084]
 [ 0.15239084  0.3149952 ]] [ 0.01113187]
[[ 0.70194817  0.09709518]
 [ 0.09709518  0.30814871]] [  8.70726362e-05]
[[ 0.7008909   0.10005269]
 [ 0.10005269  0.30040869]] [  9.66293101e-07]


### Class `GM`

`GM(L, depth, loss_fn = "MSE")` build a generative model of system size `L` and MPS depth `depth`. Trained using the loss function specified by `loss_fn`.

In [1622]:
%run "QGAN.py"
gm = GM(1, 3, loss_fn = 'JSD')

In [1623]:
for i in range(500):
    loss = gm.train()
    if i%50 == 0:
        print(gm.test())

{'h fidelity': 0.40904110837347185, 'M deviation': 0.17470390650906359}
{'h fidelity': 0.72370532049053793, 'M deviation': 0.086724218403669756}
{'h fidelity': 0.99562372370029106, 'M deviation': 0.020045222316304935}
{'h fidelity': 0.99724377729966729, 'M deviation': 0.0046568962042187979}
{'h fidelity': 0.99948108656422363, 'M deviation': 0.00094357419558209845}
{'h fidelity': 0.99969577740283455, 'M deviation': 0.00046437606099845565}
{'h fidelity': 0.99972710587171354, 'M deviation': 0.00033280008692989423}
{'h fidelity': 0.9998646142497809, 'M deviation': 0.00018074854474282704}
{'h fidelity': 0.99991146399777397, 'M deviation': 0.00013603845579194035}
{'h fidelity': 0.99992941874697427, 'M deviation': 0.00011384342417081287}


Profiling the training.

In [1060]:
%reload_ext snakeviz
%snakeviz gm.train()

 
*** Profile stats marshalled to file '/var/folders/tl/lwpcq5qj049ftcj7pvhkzv_h0000gn/T/tmpar0u4dyp'. 


In [279]:
%run "QGAN.py"
gm = GM(1, 3, loss_fn = 'MSE')
for i in range(500):
    loss = gm.train()
    if i%50 == 0:
        print(gm.test())

{'h fidelity': 0.60455973632633686, 'M deviation': 0.22034702439530796}
{'h fidelity': 0.40069881267845631, 'M deviation': 0.21627008879558482}
{'h fidelity': 0.55811284855008125, 'M deviation': 0.21448836064688032}
{'h fidelity': 0.54445063695311546, 'M deviation': 0.21157469558273778}
{'h fidelity': 0.41239827685058117, 'M deviation': 0.21467416353369759}
{'h fidelity': 0.50414573214948177, 'M deviation': 0.21288489500086116}
{'h fidelity': 0.6291191577911377, 'M deviation': 0.21126553978343868}
{'h fidelity': 0.63520073145627975, 'M deviation': 0.20564728523245329}
{'h fidelity': 0.40561063587665558, 'M deviation': 0.20576043927183713}
{'h fidelity': 0.3769370224326849, 'M deviation': 0.21282607831746619}


In [284]:
%run "QGAN.py"
mps = MPS(5, 2)
optimizer = torch.optim.SGD(mps.params, 0.01)
PM = torch.autograd.Variable(torch.Tensor(numpy.array([[1.2,0],[0,0.5]])))
for i in range(5):
    PG = mps.P(torch.ones(2))
    loss = torch.dist(PM, PG)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    mps.update()
    print(PG.data.numpy(), loss.data.numpy())

[[ 0.9999994  0.       ]
 [ 0.         0.9999994]] [ 0.53851616]
[[ 1.07681322  0.        ]
 [ 0.          0.82907784]] [ 0.35137901]
[[ 1.15965378  0.        ]
 [ 0.          0.70470172]] [ 0.20863992]
[[ 1.21114719  0.        ]
 [ 0.          0.60689586]] [ 0.1074755]
[[ 1.182181    0.        ]
 [ 0.          0.53062516]] [ 0.0354319]


In [212]:
mps.update()
mps.P(torch.ones(2))

Variable containing:
 1.3254  0.0000
 0.0000  0.4779
[torch.FloatTensor of size 2x2]

In [215]:
for unit in mps.units:
    print(unit.mat0, unit.mat)


 1.0000  0.0000
 0.0000  1.0000
[torch.FloatTensor of size 2x2]
 Variable containing:
 1.0286  0.0000
 0.0000  0.9288
[torch.FloatTensor of size 2x2]


 1.0000  0.0000
 0.0000  1.0000
[torch.FloatTensor of size 2x2]
 Variable containing:
 1.0286  0.0000
 0.0000  0.9288
[torch.FloatTensor of size 2x2]


 1.0000  0.0000
 0.0000  1.0000
[torch.FloatTensor of size 2x2]
 Variable containing:
 1.0286  0.0000
 0.0000  0.9288
[torch.FloatTensor of size 2x2]


 1.0000  0.0000
 0.0000  1.0000
[torch.FloatTensor of size 2x2]
 Variable containing:
 1.0286  0.0000
 0.0000  0.9288
[torch.FloatTensor of size 2x2]


 1.0000  0.0000
 0.0000  1.0000
[torch.FloatTensor of size 2x2]
 Variable containing:
 1.0286  0.0000
 0.0000  0.9288
[torch.FloatTensor of size 2x2]

